In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import dvu
import seaborn as sns
import os
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import imodelsx.process_results
import qa_questions
import joblib
from tqdm import tqdm
import analyze_helper
fit_encoding = __import__('01_fit_encoding')
dvu.set_style()

results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/results_apr7'
r, cols_varied, mets = analyze_helper.load_clean_results(results_dir)

In [ ]:
# imodelsx.process_results.delete_runs_in_dataframe(
#     r[r.seed != 1], actually_delete=True)

In [ ]:
d = r
# d = d[d.subject == 'UTS01']
d = d[d.feature_selection_alpha_index < 0]
d = d[d.distill_model_path == 'None']
d = d[~(d.feature_space == 'qa_embedder-10') | (d.ndelays == 8)]
d = d[~(d.feature_space == 'qa_embedder-25')]
d = d[d.pc_components == 100]
cols_varied = [c for c in cols_varied if not c in [
    'distill_model_path', 'feature_selection_alpha_index']]
# d = d[(d.qa_questions_version == 'v1') *
#   (d.qa_embedding_model == 'mistral 7B')]
if len(cols_varied) > 0:
    d = d.groupby(cols_varied)[mets].mean()
else:
    d = d[mets]

(
    d
    # .sort_values(by='corrs_test_mean', ascending=False)
    .sort_values(by='corrs_tune_pc_mean', ascending=False)
    .rename(columns=lambda x: x.replace('_', ' ').replace('corrs', ''))
    .style
    .background_gradient(cmap='magma', axis=0)
    .format(precision=3)
)

In [ ]:
# # look at distill_model_path
# d = r[(r.feature_space == 'qa_embedder-10') * (r.pc_components == 100)]
# d = d.groupby(cols_varied)[mets].mean()
# d.pivot_table(index=[c for c in cols_varied if not c == 'distill_model_path'],
#               columns='distill_model_path', values='corrs_test_mean', aggfunc='mean')

### best results breakdown

In [ ]:
r['feature_space_simplified'] = r['feature_space'].apply(
    lambda x: 'llama' if 'llama' in x else x
)
d = r
d = d[d.feature_selection_alpha_index < 0]
d = d[d.distill_model_path == 'None']
d = d[~(d.feature_space == 'qa_embedder-10') | (d.ndelays == 8)]
d = d[~(d.feature_space == 'qa_embedder-25')]
d = d[d.pc_components == 100]

d = d.sort_values(
    by='corrs_tune_pc_mean', ascending=False)
d = d.groupby(['subject', 'feature_space_simplified'])[mets]
d = d.first().reset_index()
tab = d.pivot_table(index='subject', columns='feature_space_simplified',
                    values='corrs_test_mean', aggfunc='mean')

# # add average row
tab.loc['AVG'] = tab.mean()
tab.round(3)
# for k, v in d:
#     print(k)
#     display(v.head().round(3))

In [ ]:
tab['Subject'] = tab.index.str.replace('UT', '')
# rename stuf
tab.columns = tab.columns.map(lambda x: {
    'bert-10': 'BERT',
    'eng1000': 'Eng1000',
    'llama': 'LLaMA',
    'qa_embedder-10': 'QA-Embs',
}.get(x, x))

In [ ]:
plot_data = tab.melt(id_vars='Subject',
                     var_name='feature_space_simplified', value_name='corrs_test_mean')
# fig, ax = plt.subplots(figsize=(4, 3))
plt.figure(figsize=(4.5, 3.2))
sns.barplot(
    plot_data, x='Subject', y='corrs_test_mean',
    hue='feature_space_simplified',
    hue_order=['Eng1000', 'QA-Embs', 'BERT', 'LLaMA'],
    palette=['coral', 'C0', '#777', '#333'])
# move legend outside
plt.legend(loc='upper left', frameon=False)  # , bbox_to_anchor=(0.75, 1.2))
plt.ylabel('Test correlation')
plt.ylim((0.05, 0.15))
plt.tight_layout()
plt.savefig('../figs/corr_best.pdf', bbox_inches='tight')
plt.show()

### qa version breakdowns

In [ ]:
d = r[(r.feature_space == 'qa_embedder-10') * (r.pc_components == 100)]
# d = r[(r.pc_components == 100)]
d = d[d.ndelays == 8]
d = d.groupby(cols_varied)[mets].mean()
(
    d.pivot_table(index=[c for c in cols_varied if not c == 'qa_embedding_model'],
                  columns='qa_embedding_model', values='corrs_test_mean', aggfunc='mean')
    .style.background_gradient(cmap='magma')  # , axis=0)
    .format(precision=3)
)

In [ ]:
d = r[(~r.feature_space.str.contains('qa_embedder')) * (r.pc_components == 100)]
# d = r[(r.pc_components == 100)]
# d = d[d.ndelays == 8]
d = d.groupby(cols_varied)[mets].mean()
(
    d.pivot_table(index=[c for c in cols_varied if not c == 'feature_space'],
                  columns='feature_space', values='corrs_test_mean', aggfunc='mean')
    .style.background_gradient(cmap='magma')  # , axis=0)
    .format(precision=3)
)

# Compare performance of a few different models

In [ ]:
# qa = r[(r.feature_space == 'qa_embedder-5')
# ].sort_values(by='corrs_tune_pc_mean', ascending=False).iloc[0]
qa = r.iloc[0]
eng1000 = r[(r.feature_space == 'eng1000')].sort_values(
    by='corrs_tune_pc_mean', ascending=False).iloc[0]

In [ ]:
plt.plot(qa['corrs_test'], eng1000['corrs_test'], '.', ms=1)
plt.xlabel(f'QA Embedder (mean: {qa["corrs_test"].mean():0.3f})')
plt.ylabel(f'Eng1000 (mean: {eng1000["corrs_test"].mean():0.3f})')
plt.title('Test Correlations')
m_max = max(qa['corrs_test'].max(), eng1000['corrs_test'].max())
m_min = min(qa['corrs_test'].min(), eng1000['corrs_test'].min())
plt.plot([m_min, m_max], [m_min, m_max], 'k--')
plt.show()

### Check parameters for rerunning expts (alphas, delays)

In [ ]:
args = r[(r.pc_components == -1) * (r.ndelays == 8)].iloc[0]
args = r.sort_values(by='corrs_test_mean').iloc[-1]
model_params_to_save = joblib.load(
    join(args.save_dir_unique, 'model_params.pkl'))
print(args.feature_space, args.pc_components, args.ndelays)

In [ ]:
# print which alphas are being used
pd.Series(model_params_to_save['alphas_best']).value_counts()

### Hybrid models

In [ ]:
args = r[(r.pc_components == -1) * (r.feature_space == 'qa_embedder-5')
         ].sort_values(by='corrs_tune_mean', ascending=False).iloc[0]
args2 = r[(r.pc_components > 0) * (r.feature_space == 'qa_embedder-5')
          ].sort_values(by='corrs_tune_mean', ascending=False).iloc[0]
# args = r[]

# args2 = r[(r.feature_space == 'eng1000')].iloc[0]
# args = r[(r.pc_components == -1) * (r.feature_space == 8)].iloc[0]
# args = r[(r.pc_components == -1) * (r.ndelays == 8)].iloc[0]
# model_params_to_save = joblib.load(
# join(args.save_dir_unique, 'model_params.pkl'))

In [ ]:
percentile_thresholds = np.arange(0, 100, 1)
corrs_tune_individual = args['corrs_tune']
corrs_test_individual = args['corrs_test']
corrs_test_pca = args2['corrs_test']
res = []
for percentile_threshold in percentile_thresholds:
    args_top_thresh = np.where(corrs_tune_individual > np.percentile(
        corrs_tune_individual, percentile_threshold))[0]
    args_non_top_thresh = np.where(corrs_tune_individual <= np.percentile(
        corrs_tune_individual, percentile_threshold))[0]
    args_total = np.concatenate([args_top_thresh, args_non_top_thresh])
    mean_corr_weighted = (corrs_test_individual[args_top_thresh].mean() * args_top_thresh.size +
                          corrs_test_pca[args_non_top_thresh].mean() * args_non_top_thresh.size) / args_total.size
    # print('mean corr weighted',
    #       (corrs_test_individual[args_top_thresh].mean() * args_top_thresh.size +
    #        corrs_test_pca[args_non_top_thresh].mean() * args_non_top_thresh.size) / args_total.size
    #       )
    res.append(mean_corr_weighted)
plt.plot(percentile_thresholds, res)
plt.axhline(corrs_test_individual.mean(), color='k', linestyle='--')
plt.axhline(corrs_test_pca.mean(), color='r', linestyle='--')
plt.show()

### Validate model weights usage

In [ ]:
args = r.iloc[0]
story_names_train, story_names_test = fit_encoding.get_story_names(args)
stim_test_delayed, resp_test = fit_encoding.get_data(
    args, story_names_test[:1])

In [ ]:
# test stim is all 0?
np.unique(stim_test_delayed, return_counts=True)

In [ ]:
def _calc_corrs(preds, resp):
    corrs = []
    for i in tqdm(range(preds.shape[1])):
        corrs.append(np.corrcoef(
            preds[:, i], resp[:, i])[0, 1])
    return np.array(corrs)


wt = model_params_to_save['weights']
preds_test = stim_test_delayed @ wt
corrs_test = _calc_corrs(preds_test, resp_test)
print(np.mean(corrs_test))
print(args.corrs_test_mean)
assert np.allclose(corrs_test, args['corrs_test'])

### Evaluate PC models

In [ ]:
args = r[(r.pc_components == 100) * (r.feature_space == 'qa_embedder-5')
         ].sort_values(by='corrs_tune_mean', ascending=False).iloc[0]

In [ ]:
story_names_train, story_names_test = fit_encoding.get_story_names(args)
stim_test_delayed, resp_test = fit_encoding.get_data(args, story_names_test)

In [ ]:
model_params_to_save = joblib.load(
    join(args.save_dir_unique, 'model_params.pkl'))

In [ ]:
wt = model_params_to_save['weights']
# + model_params_to_save['bias'] (not needed for just calculating corr, but needed for predictions)
preds_test = stim_test_delayed @ wt

corrs_test = _calc_corrs(preds_test, resp_test)
print(np.mean(corrs_test))
print(args.corrs_test_mean)
assert np.allclose(corrs_test, args['corrs_test'])

Original setup, before we had unpacked weights

In [ ]:
wt_pc = model_params_to_save['weights_pc']
pca = model_params_to_save['pca']
scaler_test = model_params_to_save['scaler_test']
preds_pc = stim_test_delayed @ wt_pc
preds_pc_scaled = scaler_test.inverse_transform(preds_pc)
preds_voxels = pca.inverse_transform(preds_pc_scaled)
corrs_test = _calc_corrs(preds_voxels, resp_test)
assert np.allclose(corrs_test, args['corrs_test'])

In [ ]:
# Vary number of pcs included
num_pcs = [5, 25, 50, 100, 200, 500, 1000]
corrs = []
for num_pc in num_pcs:
    wt_pc = model_params_to_save['weights_pc']
    pca = model_params_to_save['pca']
    scaler_test = model_params_to_save['scaler_test']
    preds_pc = stim_test_delayed @ wt_pc
    preds_pc_scaled = scaler_test.inverse_transform(preds_pc)

    pca_subset = deepcopy(pca)
    pca_subset.components_[num_pc:] = 0
    preds_voxels = pca_subset.inverse_transform(preds_pc_scaled)

    corrs_test = _calc_corrs(preds_voxels, resp_test)
    # assert np.allclose(corrs_test, args['corrs_test'])
    print(num_pc, np.mean(corrs_test))
    corrs.append(np.mean(corrs_test))
plt.plot(num_pcs, corrs)

In [ ]:
corrs_test = fit_encoding.evaluate_pc_model_on_each_voxel(
    args, stim_test_delayed, resp_test,
    model_params_to_save, pca, scaler_test)

In [ ]:
wt = model_params_to_save['weights']

# multiply
preds_pc = stim_test_delayed @ wt
preds_pc_unscaled = preds_pc * scaler_test.scale_ + scaler_test.mean_
preds_voxels2 = preds_pc_unscaled @ pca.components_ + pca.mean_

# rewrite the above as a multiplication of a single weight matrix
preds_voxels2 = (stim_test_delayed @ wt * scaler_test.scale_ +
                 scaler_test.mean_) @ pca.components_ + pca.mean_
weight_full = wt * scaler_test.scale_ @ pca.components_
bias_full = scaler_test.mean_ @ pca.components_ + pca.mean_
preds_voxels2 = stim_test_delayed @ weight_full + bias_full

assert np.allclose(preds_voxels, preds_voxels2)

In [ ]:
assert corrs_test.mean() == args.corrs_test_mean